## OCI Vision

Some helpful links
https://github.com/oracle/oci-python-sdk/tree/22fd62c8dbbd1aaed6b75754ec1ba8a3c16a4e5a/src/oci/ai_vision

https://docs.oracle.com/en-us/iaas/Content/vision/using/home.htm

if you have errors running sample code reach out for help in #igiu-ai-learning

Other questions #igiu-innovation-lab or #oci_ai_vision_support


## Import Libraries

In [1]:
import oci
import io
from oci.object_storage import ObjectStorageClient
import json,os


## Set input variables

In [2]:
#####
#make sure your sandbox.json file is setup for your environment. You might have to specify the full path depending on  your `cwd` 
# you can also try making your cwd ofr jupyter match your workspace python code: 
# vscopde menu -> Settings > Extensions > Jupyter > Notebook File Root
# change from ${fileDirname} to ${workspaceFolder}
#####

#SANDBOX_CONFIG_FILE = "~/work/code/python/workshop/sandbox.json"
SANDBOX_CONFIG_FILE = "sandbox.json"

FILE_TO_ANALYZE = "./vision/reciept.png"

## Read the  config file


In [3]:
scfg = None
# read the sandbox config 
with open(os.path.expanduser(SANDBOX_CONFIG_FILE), 'r') as f:
                scfg=  json.load(f)
oci_cfg = oci.config.from_file(os.path.expanduser(scfg["oci"]["configFile"]),scfg["oci"]["profile"])
bucket_cfg = scfg["bucket"]
compartmentId =scfg["oci"]["compartment"] 


## Upload file

This is an optional step. If the file is already uploaded, no need to upload it again.

In [ ]:
object_storage_client = ObjectStorageClient(oci_cfg)
print(f"Uploading file {FILE_TO_ANALYZE} ...")
object_storage_client.put_object(bucket_cfg["namespace"], 
                                bucket_cfg["bucketName"], 
                                f"{bucket_cfg['prefix']}/{os.path.basename(FILE_TO_ANALYZE)}", 
                                io.open(FILE_TO_ANALYZE,'rb'))
print("Upload completed !")

## Create AI service vision client

In [ ]:
vision_client = oci.ai_vision.AIServiceVisionClient(config=oci_cfg)

## Setup input feature for Image analysis 
You can specify the features you want to call. In the below code we are calling all the feature. comment out the features based on your input image. 

In [7]:
features = [ 
            oci.ai_vision.models.ImageClassificationFeature(),
            oci.ai_vision.models.ImageObjectDetectionFeature(),
            oci.ai_vision.models.ImageTextDetectionFeature(),
            oci.ai_vision.models.FaceDetectionFeature()
           ]

## Create Image Analysis details

In [8]:
image = oci.ai_vision.models.ObjectStorageImageDetails(
                source = "OBJECT_STORAGE",
                namespace_name =  bucket_cfg["namespace"]  ,
                bucket_name=bucket_cfg["bucketName"]  ,
                object_name=f"{bucket_cfg['prefix']}/{os.path.basename(FILE_TO_ANALYZE)}"
)


details = oci.ai_vision.models.AnalyzeImageDetails(
        features = features,
        image = image,
        compartment_id=compartmentId
    )

## Analysize the image  


In [ ]:
response = vision_client.analyze_image (details)

print(response.data)